In [2]:
import torch
from torch_geometric.nn import GCNConv
from torch.nn import Linear
from torch_geometric.nn import global_mean_pool
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from pool_class import *
from torch_geometric.transforms import NormalizeFeatures

In [3]:
dataset = Planetoid(root='data/Planetoid', name='PubMed', transform=NormalizeFeatures())
n = (len(dataset) + 9) // 10
data = dataset[0]

Processing...
Done!


Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])

In [ ]:
# Test whether the message can flow (gradient can be updated automatically)
class dispooling_GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        torch.manual_seed(1234)
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.pool1 = pooling(hidden_channels, score_method=2, p1=0.3, p2=0.95, aggregate_score_method='avg')
        # self.poo1 = EdgePooling(hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.pool2 = pooling(hidden_channels, score_method=2, p1=0.3, p2=0.95, aggregate_score_method='avg')

        self.conv3 = GCNConv(hidden_channels, hidden_channels)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index).relu()
        edge_index, _, h, _, _ = self.pool1(edge_index, h)
        # h, edge_index, _, _ = self.pool1(x, edge_index, batch=None)
        h = self.conv2(h, edge_index).relu()
        edge_index, _, h, _, _ = self.pool2(edge_index, h)
        h = self.conv3(h, edge_index).relu()



        h = global_mean_pool(h, batch=None)
        return F.log_softmax(h, dim=-1)

In [ ]:
model = dispooling_GCN(in_channels=dataset.num_node_features, hidden_channels=20, out_channels=dataset.num_classes)
model.forward(data.x, data.edge_index)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    for data in train_dataset:
        optimizer.zero_grad() 
        out = model(data.x, data.edge_index)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
    return(loss)


def test():
    model.eval()
    correct = 0
    for data in test_dataset:
        pred = model(data.x, data.edge_index).argmax(dim=1)
        correct += int((pred == data.y).sum())
    return correct / len(test_dataset)


def val():
    model.eval()
    correct = 0
    for data in val_dataset:
        pred = model(data.x, data.edge_index).argmax(dim=1)
        correct += int((pred == data.y).sum())
    return correct / len(test_dataset)



best_val_acc = 0
for epoch in range(1, 171):
    train_loss = train()
    val_acc = val()
    if val_acc > best_val_acc:
        test_acc = test()
        best_val_acc = val_acc
    if epoch % 10 == 1:
        print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, '
            f'Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')